In [ ]:
import os, sys

import pandas as pd
import numpy as np

from importlib import reload

from data_warehouse_utils.dataloader import DataLoader

In [ ]:
os.chdir('/home/adam/adam/causal_inference')

from causal_inference.create_experiment.get_proning_sessions import UseCaseLoader

reload(sys.modules['causal_inference.create_experiment.get_proning_sessions'])
reload(sys.modules['causal_inference.create_experiment.create_observations'])
reload(sys.modules['causal_inference.create_experiment.create_covariates'])
reload(sys.modules['causal_inference.create_experiment.create_outcome'])

from causal_inference.create_experiment.get_proning_sessions import UseCaseLoader

In [ ]:
os.chdir('/home/adam/adam/data/13012020/')
df = pd.read_csv('use_case_inclusion_8h_all_outputs_cvvh.csv')
df.start_timestamp = df.start_timestamp.astype('datetime64[ns]')
df.end_timestamp = df.end_timestamp.astype('datetime64[ns]')
df.info(max_cols = 200)


In [ ]:
os.chdir('/home/adam/adam/data/19012021/')
df_measurements = pd.read_csv('pf_measurements_all.csv')
df_measurements.effective_timestamp = df_measurements.effective_timestamp.astype('datetime64[ns]')
df_measurements.info(max_cols = 200)
df_measurements = df_measurements.sort_values(by='effective_timestamp', ascending=True)

In [ ]:
df_measurements.head()

In [ ]:
dl = UseCaseLoader()

In [ ]:
df = dl.add_outcomes(df=df, df_measurements=df_measurements)

In [ ]:
df.info(max_cols=200)

In [ ]:
df.iloc[:, -4:]

In [ ]:
os.chdir('/home/adam/adam/data/19012021/')
df.to_csv('observational_data_8h_inclusion_all_outputs_1.csv', index=False)

In [ ]:
os.chdir('/home/adam/adam/data/19012021/')
df = pd.read_csv('observational_data_8h_inclusion_all_outputs_1.csv')
df.start_timestamp = df.start_timestamp.astype('datetime64[ns]')
df.end_timestamp = df.end_timestamp.astype('datetime64[ns]')
df.info(max_cols = 200)

In [ ]:
df = df[df['fio2'] > 60]
df['diff'] = (df.loc[:, "pf_ratio_2h_8h_outcome"] - df.loc[:, "pf_ratio_2h_8h_manual_outcome"]).round(2)

In [ ]:
df['diff'].describe().round()

In [ ]:
df_outlier = df[df['diff'] < -100]
df_outlier.info()
df_outlier['pf_ratio'] = df_outlier['po2'] / df_outlier['fio2']
df_outlier['pf_ratio'] = df_outlier['pf_ratio'] * 100
df_outlier['pf_ratio'] = df_outlier['pf_ratio'].round(2)

In [ ]:
df_outlier = df_outlier[df_outlier['pf_ratio'] < 150]

df_outlier.loc[:, ['hash_session_id',
                   'start_timestamp',
                   'end_timestamp',
                   'duration_hours',
                   'po2',
                   'fio2',
                   "pf_ratio_2h_8h_outcome",
                   "pf_ratio_2h_8h_manual_outcome"]]

In [ ]:
from datetime import timedelta

dl = DataLoader()

parameters = ['pao2_over_fio2',
              'po2_arterial',
              'po2_unspecified',
              'fio2',
              'po2_arterial',
              'po2_unspecified_over_fio2']

In [ ]:
patient = df_outlier.hash_patient_id.iloc[0]
start = df_outlier.start_timestamp.iloc[0] + timedelta(hours=2)
end = df_outlier.start_timestamp.iloc[0] + timedelta(hours=8)

signal = dl.get_single_timestamp(parameters=parameters,
                                 patients=[patient],
                                 columns=['hash_patient_id',
                                          'effective_timestamp',
                                          'effective_value',
                                          'pacmed_name'])

signal = signal[(signal.effective_timestamp >= start) & (signal.effective_timestamp <= end)]
signal


In [ ]:
patient = df_outlier.hash_patient_id.iloc[1]
start = df_outlier.start_timestamp.iloc[1] + timedelta(hours=2)
end = df_outlier.start_timestamp.iloc[1] + timedelta(hours=8)

signal = dl.get_single_timestamp(parameters=parameters,
                                 patients=[patient],
                                 columns=['hash_patient_id',
                                          'effective_timestamp',
                                          'effective_value',
                                          'pacmed_name', 'pacmed_subname'])

signal = signal[(signal.effective_timestamp >= start) & (signal.effective_timestamp <= end)]
signal

In [ ]:
patient = df_outlier.hash_patient_id.iloc[2]
start = df_outlier.start_timestamp.iloc[2] + timedelta(hours=2)
end = df_outlier.start_timestamp.iloc[2] + timedelta(hours=8)

signal = dl.get_single_timestamp(parameters=parameters,
                                 patients=[patient],
                                 columns=['hash_patient_id',
                                          'effective_timestamp',
                                          'effective_value',
                                          'pacmed_name'])

signal = signal[(signal.effective_timestamp >= start) & (signal.effective_timestamp <= end)]
signal

In [ ]:
patient = df_outlier.hash_patient_id.iloc[3]
start = df_outlier.start_timestamp.iloc[3] + timedelta(hours=2)
end = df_outlier.start_timestamp.iloc[3] + timedelta(hours=16)

signal = dl.get_single_timestamp(parameters=parameters,
                                 patients=[patient],
                                 columns=['hash_patient_id',
                                          'effective_timestamp',
                                          'effective_value',
                                          'numerical_value',
                                          'pacmed_name',
                                          'pacmed_subname'])

signal = signal[(signal.effective_timestamp >= start) & (signal.effective_timestamp <= end)]
signal

In [ ]:
df_measurements = signal

df_measurements.loc[df_measurements.pacmed_name == 'po2_unspecified', 'pacmed_name'] = 'po2_arterial' #change

po2 = df_measurements[df_measurements.pacmed_name == 'po2_arterial']
po2 = po2['numerical_value']
fio2 = df_measurements[df_measurements.pacmed_name == 'fio2']
fio2 = fio2['numerical_value']

if (len(po2) > 0) & (len(fio2) > 0):
        # this is a bug as we take the last value of po2 and fio2 as the pf ratio. Should do the same with po2_over_fio2
    po2 = po2.iloc[-1]
    fio2 = fio2.iloc[-1]
    pf_ratio = round((po2 / fio2) * 100)
else:
    pf_ratio = np.NaN

In [ ]:
pf_ratio

In [ ]:
patient = df_outlier.hash_patient_id.iloc[4]
start = df_outlier.start_timestamp.iloc[4] + timedelta(hours=2)
end = df_outlier.start_timestamp.iloc[4] + timedelta(hours=8)

signal = dl.get_single_timestamp(parameters=parameters,
                                 patients=[patient],
                                 columns=['hash_patient_id',
                                          'effective_timestamp',
                                          'effective_value',
                                          'pacmed_name'])

signal = signal[(signal.effective_timestamp >= start) & (signal.effective_timestamp <= end)]
signal

In [ ]:
dl = UseCaseLoader()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.distplot(df_outlier['po2'],
              hist = True,
             kde = True,
             label='pao2_over_fio2',
             bins=50)

In [ ]:
sns.distplot(df_outlier['fio2'],
            hist = True,
            kde = True,
            label='manual pf_ratio',
            bins=50)

In [ ]:
sns.distplot(df_outlier['pf_ratio'],
            hist = True,
            kde = True,
            label='manual pf_ratio',
            bins=50)


In [ ]:
plt.legend(prop={'size': 12})
plt.xlabel('pf')
plt.ylabel('Density')
plt.xlim(left=0, right=500)

In [ ]:
sns.distplot(df_outlier['pf_ratio'],
            hist = True,
            kde = True,
            label='pf_ratio_covariate',
            bins=50)

sns.distplot(df['pf_ratio_2h_8h_outcome'],
              hist = True,
             kde = True,
             label='pao2_over_fio2',
             bins=50)

sns.distplot(df['pf_ratio_2h_8h_manual_outcome'],
            hist = True,
            kde = True,
            label='manual pf_ratio',
            bins=50)

plt.legend(prop={'size': 12})
plt.xlabel('pf')
plt.ylabel('Density')
plt.xlim(left=0, right=500)

In [ ]:
sns.distplot(df['pf_ratio_12h_24h_outcome'],
              hist = True,
             kde = True,
             label='pao2_over_fio2',
             bins=50)

sns.distplot(df['pf_ratio_12h_24h_manual_outcome'],
            hist = True,
            kde = True,
            label='manual pf_ratio',
            bins=50)

plt.legend(prop={'size': 12})
plt.xlabel('pf')
plt.ylabel('Density')
plt.xlim(left=0, right=500)


In [ ]:
df_plot = df.loc[df['pf_ratio_12h_24h_outcome'].isna(),'pf_ratio_12h_24h_manual_outcome']
print(df_plot.mean())
sns.distplot(df_plot,
              hist = True,
             kde = True,
             label='pao2_over_fio2',
             bins=50)

df_plot = df.loc[~df['pf_ratio_12h_24h_outcome'].isna(),'pf_ratio_12h_24h_manual_outcome']
print(df_plot.mean())
sns.distplot(df_plot,
            hist = True,
            kde = True,
            label='manual pf_ratio',
            bins=50)

plt.legend(prop={'size': 12})
plt.xlabel('pf')
plt.ylabel('Density')
plt.xlim(left=0, right=500)

In [ ]:
df['diff'] = (df.iloc[:, 0] - df.iloc[:, 1]).abs().round(2)

In [ ]:
diff['diff'].describe(percentiles=[.5, .6, .7, .8, .9])

In [ ]:

df_diff = df[['artificial_session',
              'fio2_inclusion_8h',
              'fio2',
              'peep_inclusion_8h',
              'peep',
              'po2_inclusion_8h',
              'po2']]

In [ ]:

df_diff['fio2_diff'] = df_diff['fio2_inclusion_8h'] - df_diff['fio2']
df_diff['peep_diff'] = df_diff['peep_inclusion_8h'] - df_diff['peep']
df_diff['po2_diff'] = df_diff['po2_inclusion_8h'] - df_diff['po2']

In [ ]:
df_diff[df_diff.artificial_session == False].describe()